# Zarr (v3) interoperability between Python and C++

In this notebook we use [zarrita](https://github.com/alimanfoo/zarrita) and [xtensor-zarr](https://github.com/xtensor-stack/xtensor-zarr) to make use of a common Zarr store interchangeably: a Zarr store written by zarrita can be read by xtensor-zarr, and the other way around.

## Download development source code

In [ ]:
>bash
rm -rf zarrita xtensor xtensor-io xtl *.tar.gz*
mkdir -p zarrita xtensor xtensor-io xtl

wget https://github.com/davidbrochart/zarrita/archive/codec.tar.gz -O zarrita.tar.gz -q
tar zxf zarrita.tar.gz -C zarrita --strip-components 1
mv zarrita/zarrita.py .

wget https://github.com/davidbrochart/xtensor/archive/zarray.tar.gz -O xtensor.tar.gz -q
tar zxf xtensor.tar.gz -C xtensor --strip-components 1

wget https://github.com/xtensor-stack/xtensor-io/archive/master.tar.gz -O xtensor-io.tar.gz -q
tar zxf xtensor-io.tar.gz -C xtensor-io --strip-components 1

wget https://github.com/xtensor-stack/xtl/archive/master.tar.gz -O xtl.tar.gz -q
tar zxf xtl.tar.gz -C xtl --strip-components 1

In [ ]:
>xcpp14
#pragma cling add_include_path("xtensor/include")
#pragma cling add_include_path("xtensor-io/include")
#pragma cling add_include_path("xtl/include")
#pragma cling add_include_path("../include")

In [ ]:
>xcpp14
#include "xtensor/xview.hpp"
#include "xtensor/xchunked_array.hpp"
#include "xtensor/xchunk_store_manager.hpp"
#include "xtensor/xfile_array.hpp"
#include "xtensor/xdisk_io_handler.hpp"
#include "xtensor-io/xio_binary.hpp"
#include "xtensor-zarr/xzarr_hierarchy.hpp"
#include "xtensor-zarr/xzarr_file_system_store.hpp"

## Create a Zarr hierarchy and array in Python

In [ ]:
>bash
rm -rf h_zarrita.zr3

In [ ]:
>python3
import zarrita
import numpy as np

h = zarrita.create_hierarchy('h_zarrita.zr3')
attrs = {'question': 'life', 'answer': 42}
a = h.create_array('/arthur/dent', shape=(5, 10), dtype='float64', chunk_shape=(2, 5), compressor=None, attrs=attrs)
a[:, :] = np.arange(5 * 10).reshape(5, 10)

In [ ]:
>bash
find h_zarrita.zr3

In [ ]:
>bash
cat h_zarrita.zr3/meta/root/arthur/dent.array

## Open the Zarr hierarchy and array in C++

In [ ]:
>xcpp14
xt::xzarr_file_system_store s("h_zarrita.zr3");
auto h = xt::get_zarr_hierarchy(s);
xt::zarray z = h.get_array("/arthur/dent");

## Create a Zarr hierarchy and array in C++

In [ ]:
>xcpp14
std::vector<size_t> shape = {4, 4};
std::vector<size_t> chunk_shape = {2, 2};
nlohmann::json attrs = {{"question", "life"}, {"answer", 42}};
xt::xzarr_file_system_store s("h_xtensor.zr3");
auto h = xt::create_zarr_hierarchy(s);
xt::zarray z = h.create_array("/arthur/dent", shape, chunk_shape, "<f8", '/', xt::xio_binary_config(), attrs);

In [ ]:
>bash
find h_xtensor.zr3

In [ ]:
>bash
cat h_xtensor.zr3/meta/root/arthur/dent.array

## Open the Zarr hierarchy and array in Python

In [ ]:
>python3
h = zarrita.get_hierarchy('h_xtensor.zr3')
a = h['arthur/dent']

assert a.shape == (4, 4)
assert a.dtype == np.dtype('float64')
assert a.chunk_shape == (2, 2)
assert a.compressor == None
assert a.attrs == {'answer': 42, 'question': 'life'}